In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import DLRCP.theoreticalAnalysis as theoTool

%matplotlib inline

In [2]:
beta=0.6
alpha = 2

beta_digit1 = int(beta) % 10
beta_digit2 = int(beta*10) % 10
beta_digit3 = int(beta*100) % 10

QDebugFile = 'Results/case_study_TimeDiscount_alpha_2_0/TimeDiscount_{}_{}_{}/QTable.txtdebug.csv'.format(beta_digit1, beta_digit2, beta_digit3)
QTableFile = 'Results/case_study_TimeDiscount_alpha_2_0/TimeDiscount_{}_{}_{}/QTable.txt'.format(beta_digit1, beta_digit2, beta_digit3)
df = pd.read_csv(QDebugFile)
QTable = pd.read_csv(QTableFile, names=['a=0', 'a=1'])

In [3]:
delays_exp = df['curDelay'].mean()
deliveryRate_exp = df['curDelvyRate'].mean()
delays_lst25_exp = df['curDelay'][-1000:].mean()
deliveryRate_lst25_exp = df['curDelvyRate'][-1000:].mean()

gamma_exp = df['curChPktLoss'].mean()
txAttempts = df['curTxAttemp']
RTT_exp = df['curRTT'].mean()
RTO_exp = df['curRTO'].mean()
RTTVar_exp = df['curRTTVar'].mean()

In [4]:
def utilityCalcHandler(delvyRate, avgDelay):
    return beta ** (avgDelay / 100) * (delvyRate ** alpha)

In [5]:
# calculate s from QTable
s_exp = (QTable['a=1'] > QTable['a=0']).sum()

# channelPktLossRate = 0.3
# channelDelay = 125
# channelRTO = 125 + 50/4 * 4

channelPktLossRate = gamma_exp
channelDelay = RTT_exp
# channelRTO = RTO_exp
channelRTO = RTO_exp

smax=30
utility_list = []
delay_list = []
delvy_list = []
for rx in range(1, smax):
    avgDelay = theoTool.calc_delay_expect(channelPktLossRate, channelDelay, channelRTO, rx)
    avgDeliveryRate = theoTool.calc_delvy_rate_expect(channelPktLossRate, rx)
    
    delvy_list.append(avgDeliveryRate)
    delay_list.append(avgDelay)
    utility_list.append(utilityCalcHandler(
        delvyRate=avgDeliveryRate,
        avgDelay=avgDelay,
    ))


s_theory = np.argmax(utility_list) + 1 # because rx starts from 1
best_s_list = ['' if s != s_theory else 'best' for s in range(1, smax) ]

utility_for_each_s = pd.DataFrame({'s': list(range(1, smax)), 'delay': delay_list, 'delivery rate': delvy_list, 'utility': utility_list, 'best s': best_s_list})
utility_for_each_s

,s,delay,delivery rate,utility,best s
0,1,123.971198,0.700829,0.260733,
1,2,164.306766,0.910497,0.358133,
2,3,184.285967,0.973223,0.369478,best
3,4,193.085711,0.991989,0.366991,
4,5,196.639745,0.997603,0.364479,
5,6,197.989657,0.999283,0.363195,
6,7,198.480665,0.999785,0.362649,
7,8,198.653798,0.999936,0.362438,
8,9,198.713462,0.999981,0.362360,
9,10,198.733667,0.999994,0.362332,


In [6]:
# delay and delivery rate from data
delays_exp, deliveryRate_exp, delays_lst25_exp, deliveryRate_lst25_exp

(173.59842282384852, 0.8670640665025869, 176.047, 0.876057)

In [7]:
# v from theory

# RTQ_v, _ = theoTool.calc_V_theo_norm(RTQ_pktLossRate, timeDivider, alpha, beta, RTQ_delay, RTQ_delay_var, maximumSimulationSmax)
# RTQ_v, _ = theoTool.calc_V_theo_uniform(RTQ_pktLossRate, timeDivider, alpha, beta, RTQ_delay-2*RTQ_delay_var, RTQ_delay+2*RTQ_delay_var, maximumSimulationSmax)

v_theory, s_theory = theoTool.calc_V_theo_norm(gamma_exp, 100, alpha, beta, RTT_exp, RTTVar_exp, smax)
# {'Q - theory': v_theory, 'Q-exp': QTable[:, 1]}
q_theory = np.zeros((smax, 2))
q_theory[:, 0] = v_theory[s_theory]
q_theory[:s_theory, 1] = v_theory[:s_theory]
s_theory_list = ['' if s != (s_theory-1) else '*' for s in range(0, smax) ]
exp_vs_theory_v = pd.DataFrame({'Q-theory:a=0': q_theory[:, 0], 'Q-theory:a=1': q_theory[:, 1], 'Q-theory: s*': s_theory_list})

v_exp = QTable
v_exp.reset_index()
s_exp_list = ['' if s != (s_exp-1) else '*' for s in range(0, smax) ]


exp_vs_theory_v = pd.concat([exp_vs_theory_v, v_exp, pd.DataFrame({'Q-exp: s*':s_exp_list})], ignore_index=False, axis=1)
exp_vs_theory_v.rename(
    columns={
        'a=0': 'Q-exp: a=0',
        'a=1': 'Q-exp: a=1'
    }
)
# exp_vs_theory_v.dropna(how='any')
# v_exp

,Q-theory:a=0,Q-theory:a=1,Q-theory: s*,Q-exp: a=0,Q-exp: a=1,Q-exp: s*
0,0.369469,0.393766,,0.330885,0.705373,
1,0.369469,0.178028,,0.330885,0.344733,*
2,0.369469,0.212887,*,0.330885,0.238833,
3,0.369469,0.000000,,0.330885,0.036511,
4,0.369469,0.000000,,NaN,NaN,
5,0.369469,0.000000,,NaN,NaN,
6,0.369469,0.000000,,NaN,NaN,
7,0.369469,0.000000,,NaN,NaN,
8,0.369469,0.000000,,NaN,NaN,
9,0.369469,0.000000,,NaN,NaN,
